We talked about a simple way to parallel your python code by using joblib in [a former blog](http://qingkaikong.blogspot.com/2016/06/python-module-joblib-make-parallelism.html). Today, I want to use it to parallel a method in a class, but I encountered some problem. In this week's blog, I will show you how we can solve the problem by using the joblib (You can use python's [multiprocessing](https://docs.python.org/2/library/multiprocessing.html) as well). 

## Error! If we parallel directly

In [32]:
from joblib import Parallel, delayed

In [53]:
# let's re-write the exact function before into a class
class square_class:
    def square_int(self, i):
        return i * i
     
    def run(self, num):
        results = []
        results = Parallel(n_jobs= -1, backend="threading")\
            (delayed(self.square_int)(i) for i in num)
        print(results)

In [54]:
square_int = square_class()
square_int.run(num = range(10))

TypeError: can't pickle instancemethod objects

## Solution to the problem

Why we got the above error? 

joblib or Multiprocessing is based on pickling to pass functions around to achieve parallelization. In order to pickle the object, this object must capable of being referred to in the global context for the unpickle to be able to access it. The function we want to parallel above is not in global context, therefore, causing the error.  

Therefore, one solution I found [online](http://www.rueckstiess.net/research/snippets/show/ca1d7d90) is to create a function outside the class to unpack the self from the arguments and calls the function again. Here's the example:  

In [51]:
def unwrap_self(arg, **kwarg):
    return square_class.square_int(*arg, **kwarg)

class square_class:
    def square_int(self, i):
        return i * i
     
    def run(self, num):
        results = []
        results = Parallel(n_jobs= -1, backend="threading")\
            (delayed(unwrap_self)(i) for i in zip([self]*len(num), num))
        print(results)

In [52]:
square_int = square_class()
square_int.run(num = range(10))

[0, 1, 4, 9, 16]


We can see from above, by simply write a function outside of the class, we can parallel the method in class as before. You may wonder what we passed to the function, if we print out the argument we passed to unwrap_self function, you will have a better understanding:   

In [56]:
num = range(10)
# note, I use a string "self" here to print it out
print(zip(["self"]*len(num), num))

[('self', 0), ('self', 1), ('self', 2), ('self', 3), ('self', 4), ('self', 5), ('self', 6), ('self', 7), ('self', 8), ('self', 9)]
